In [19]:
print('Hello')

Hello


In [25]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path='../credential/Xpring Dev Sandbox-edb2a98acf17.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [26]:
key_path

'../credential/Xpring Dev Sandbox-edb2a98acf17.json'

In [27]:
import os.path
from os import path

path.exists(key_path)

True

In [28]:
def gbq_query(query,cred, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(credentials=cred)
    job_config = bigquery.QueryJobConfig()
    return client.query(query, job_config=job_config).to_dataframe()

In [29]:
query = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"
AND TxnSignature IS NOT NULL
AND Destination IS NOT NULL
AND AmountXRP  IS NOT NULL
AND TIMESTAMP(l.CloseTime) >=TIMESTAMP('2019-12-15')
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP('2019-12-20')


ORDER BY TimeStamp
LIMIT 100000;
"""


In [30]:
xrp = gbq_query(query,credentials)
xrp

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex
0,2019-12-15 00:00:00+00:00,rPsmHDMkheWZvbAkTA8A9bVnUdadPn7XBK,rUMhAy8zECaY9u6ntqTAXFu1t8gixCeHka,10,304502210089A1985CB7AF1F667C61405FB4A609F06DEB...,48560300000,52058308
1,2019-12-15 00:00:01+00:00,rw2ciyaNshpHe7bCHo4bRWq6pqqynnWKQg,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,40,3045022100AEF609203571841221F6D13CD5991EE66AAF...,50000083642,52058309
2,2019-12-15 00:00:01+00:00,rEvoRfi83Hc5gxqpwRtc9jh7Z4C2hECCWY,rnuPTVikw8HKK4hBGCtnq2J2433VYaZPZQ,12,304402203E8C80A7E4F36EEB754BCA9EF3CB2D5B2BFA08...,999999,52058309
3,2019-12-15 00:00:10+00:00,ryBANkk28Mj71jRKAkt13U1X9ubztsGWZ,rwU8rAiE2eyEPz3sikfbHuqCuiAtdXqa2v,7707,304402203D711CFEB876FFC22375F0C8C3CC3B1DB8952D...,15728776000,52058310
4,2019-12-15 00:00:11+00:00,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,rwpMvfxoodXggJ1g4qv6MWAPQqWDwQyHUW,40,3045022100F4BEBE96EF6610512BD07D789C1E2629342F...,1162250116,52058311
...,...,...,...,...,...,...,...
99995,2019-12-19 06:12:22+00:00,rUtCyL45mkqkSqvnoGMHJsqopmrAr1azDa,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,3045022100EACEFB09AD26111502E19898D01087C7E631...,2664,52151549
99996,2019-12-19 06:12:30+00:00,rUtCyL45mkqkSqvnoGMHJsqopmrAr1azDa,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,3045022100FA7E5E19D1FEA7E0FFB3FB828A784BFCACC0...,2664,52151550
99997,2019-12-19 06:12:30+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,30440220667E43D2230337A0ACBB7FC4278D6170B1ABA9...,300,52151550
99998,2019-12-19 06:12:30+00:00,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,rGsBXMnVZ4PdfsDTB2r7qGFzQAtfjiBLS1,40,30440220297A12429D6EE005FC5D50AA012A2E406BE9A2...,1419675959,52151550
